In [1]:

import pandas as pd
import tqdm

header = {
    "User-Agent": "harshit harshit.gola.off@gmail.com",
}


In [2]:
# Let's convert CIK JSON to pandas DataFrame
# First load the data into python dictionary


CIK_df=pd.read_json("/home/jupyter/SEC_exctractor/company_tickers.json").T

In [3]:
CIK_df.columns = ['cik', 'ticker', 'title']

In [4]:
df_history = pd.read_csv("/home/jupyter/SEC_exctractor/data/history.csv")

/tmp/ipykernel_2102/571638552.py:1: DtypeWarning: Columns (4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_history = pd.read_csv("/home/jupyter/SEC_exctractor/data/history.csv")


In [5]:
df_history

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,core_type,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription,ticker,cik
0,0000018230-25-000009,2025-02-14,NaN,2025-02-14T09:38:46.000Z,34,IRANNOTICE,001-00768,25623988.0,NaN,IRANNOTICE,76041,0,0,catirannotice-202410xk.htm,IRANNOTICE,CAT,18230
1,0000018230-25-000008,2025-02-14,2024-12-31,2025-02-14T09:36:30.000Z,34,10-K,001-00768,25623971.0,NaN,XBRL,33945394,1,1,cat-20241231.htm,10-K,CAT,18230
2,0001104659-25-012855,2025-02-13,2025-02-11,2025-02-13T14:41:51.000Z,NaN,4,NaN,NaN,NaN,4,8873,0,0,xslF345X05/tm255985-10_4seq1.xml,OWNERSHIP DOCUMENT,CAT,18230
3,0001104659-25-012854,2025-02-13,2025-02-11,2025-02-13T14:40:49.000Z,NaN,4,NaN,NaN,NaN,4,6563,0,0,xslF345X05/tm255985-9_4seq1.xml,OWNERSHIP DOCUMENT,CAT,18230
4,0001104659-25-012853,2025-02-13,2025-02-11,2025-02-13T14:39:50.000Z,NaN,4,NaN,NaN,NaN,4,6513,0,0,xslF345X05/tm255985-8_4seq1.xml,OWNERSHIP DOCUMENT,CAT,18230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142248,0001181431-14-022493,2014-06-03,2014-06-01,2014-06-03T18:30:40.000Z,NaN,4,NaN,NaN,NaN,4,8644,0,0,xslF345X03/rrd410741.xml,2014.06.01 AHRENDTS FORM 4 - RSU VESTING,AAPL,320193
142249,0001181431-14-021968,2014-05-30,2014-05-29,2014-05-30T18:31:49.000Z,NaN,4,NaN,NaN,NaN,4,9911,0,0,xslF345X03/rrd410500.xml,2014.05.29 MAESTRI FORM 4 - RSU GRANT,AAPL,320193
142250,0001181431-14-021967,2014-05-30,2014-05-28,2014-05-30T18:31:09.000Z,NaN,4,NaN,NaN,NaN,4,11247,0,0,xslF345X03/rrd410403.xml,14.05.28 RICCIO FORM 4 - 10B5-1 SALE,AAPL,320193
142251,0001181431-14-021923,2014-05-30,2014-05-29,2014-05-30T17:23:51.000Z,34.0,8-K,000-10030,14881425.0,5.02,8-K,13670,0,0,rrd410610.htm,"CURRENT REPORT, ITEM 5.02",AAPL,320193


In [6]:
CIK_df

,cik,ticker,title
0,320193,AAPL,Apple Inc.
1,789019,MSFT,MICROSOFT CORP
2,1652044,GOOGL,Alphabet Inc.
3,1018724,AMZN,AMAZON COM INC
4,1045810,NVDA,NVIDIA CORP
...,...,...,...
9236,1849280,GSQB-WT,"G Squared Ascend II, Inc."
9237,1735707,GTXAP,Garrett Motion Inc.
9238,1813914,CMAXW,"CareMax, Inc."
9239,1867287,LBBBW,Lakeshore Acquisition II Corp.


In [10]:


#!/usr/bin/env python
# coding: utf-8

# # 10-K form
# ## Business, Risk, and MD&A
# The function *parse_10k_filing()* parses 10-K forms to extract the following sections: business description, business risk, and management discussioin and analysis. The function takes two arguments, a link and a number indicating the section, and returns a list with the requested sections. Current options are **0(All), 1(Business), 2(Risk), 4(MDA).**
# 
# Caveats:
# The function *parse_10k_filing()* is a parser. You need to feed a SEC text link into it. There are many python and r packages to get a direct link to the fillings.
# 
import re
import unicodedata
import pandas as pd
from bs4 import BeautifulSoup as bs
from tqdm import tqdm

def parse_10k_filing(file_path, section):
    
    if section not in [0, 1, 2, 3]:
        print("Not a valid section")
        sys.exit()
    
    def get_text(file_path):
        with open(file_path, 'r') as file:
            content = file.read()
        html = bs(content, 'html.parser')
        text = html.get_text()
        text = unicodedata.normalize("NFKD", text).encode('ascii', 'ignore').decode('utf8')
        text = text.split("\n")
        text = " ".join(text)
        return text
    
    def extract_text(text, item_start, item_end):
        item_start = item_start
        item_end = item_end
        starts = [i.start() for i in item_start.finditer(text)]
        ends = [i.start() for i in item_end.finditer(text)]
        positions = list()
        for s in starts:
            control = 0
            for e in ends:
                if control == 0:
                    if s < e:
                        control = 1
                        positions.append([s,e])
        item_length = 0
        item_position = list()
        for p in positions:
            if (p[1]-p[0]) > item_length:
                item_length = p[1]-p[0]
                item_position = p

        item_text = text[item_position[0]:item_position[1]]

        return item_text

    text = get_text(file_path)
        
    if section == 1 or section == 0:
        try:
            item1_start = re.compile("item\s*[1][\.\;\:\-\_]*\s*\\b", re.IGNORECASE)
            item1_end = re.compile("item\s*1a[\.\;\:\-\_]\s*Risk|item\s*2[\.\,\;\:\-\_]\s*Prop", re.IGNORECASE)
            businessText = extract_text(text, item1_start, item1_end)
        except:
            businessText = "Something went wrong!"
        
    if section == 2 or section == 0:
        try:
            item1a_start = re.compile("(?<!,\s)item\s*1a[\.\;\:\-\_]\s*Risk", re.IGNORECASE)
            item1a_end = re.compile("item\s*2[\.\;\:\-\_]\s*Prop|item\s*[1][\.\;\:\-\_]*\s*\\b", re.IGNORECASE)
            riskText = extract_text(text, item1a_start, item1a_end)
        except:
            riskText = "Something went wrong!"
            
    if section == 3 or section == 0:
        try:
            item7_start = re.compile("item\s*[7][\.\;\:\-\_]*\s*\\bM", re.IGNORECASE)
            item7_end = re.compile("item\s*7a[\.\;\:\-\_]\sQuanti|item\s*8[\.\,\;\:\-\_]\s*", re.IGNORECASE)
            mdaText = extract_text(text, item7_start, item7_end)
        except:
            mdaText = "Something went wrong!"
    
    if section == 0:
        data = [businessText, riskText, mdaText]
    elif section == 1:
        data = [businessText]
    elif section == 2:
        data = [riskText]
    elif section == 3:
        data = [mdaText]
    
    return data




def parse_all_forms(df, form, header):
    df_ = df[df.form == form]
    df__ = pd.DataFrame()
    for index, row in tqdm(df_.iterrows(), total=df_.shape[0]):
        file_path = f"/home/jupyter/SEC_exctractor/data/{row['ticker']}/{row['primaryDocument']}"
        section = 0
        
        # Parse the 10-K filing and store the results in a DataFrame
        text_data = parse_10k_filing(file_path, section)
        df_text = pd.DataFrame({'Text': text_data})
        df_text['ticker'] = row['ticker']
        df_text['filepath'] = file_path
        df__ = pd.concat([df_text, df__])

    return df__



df_text = parse_all_forms(df_history , '10-K', header)


## Exporting the resulted dataframe in a csv format.

file_path='./All Reports Data.csv'


#Arranging the index and adding Id column
df_text = df_text.reset_index(drop=True)
df_text['Id'] = df_text.index+1

# Pop the column from its current position
new_column = df_text.pop('Id')

# Insert the column at the front of the DataFrame
df_text.insert(0, 'Id', new_column)

df_text.to_csv(file_path, index=False)



  0%|          | 0/540 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/home/jupyter/SEC_exctractor/data/CAT/cat-20241231.htm'

In [14]:
df_history[df_history['ticker']=="CAT"]

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,core_type,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription,ticker,cik
0,0000018230-25-000009,2025-02-14,NaN,2025-02-14T09:38:46.000Z,34,IRANNOTICE,001-00768,25623988.0,NaN,IRANNOTICE,76041,0,0,catirannotice-202410xk.htm,IRANNOTICE,CAT,18230
1,0000018230-25-000008,2025-02-14,2024-12-31,2025-02-14T09:36:30.000Z,34,10-K,001-00768,25623971.0,NaN,XBRL,33945394,1,1,cat-20241231.htm,10-K,CAT,18230
2,0001104659-25-012855,2025-02-13,2025-02-11,2025-02-13T14:41:51.000Z,NaN,4,NaN,NaN,NaN,4,8873,0,0,xslF345X05/tm255985-10_4seq1.xml,OWNERSHIP DOCUMENT,CAT,18230
3,0001104659-25-012854,2025-02-13,2025-02-11,2025-02-13T14:40:49.000Z,NaN,4,NaN,NaN,NaN,4,6563,0,0,xslF345X05/tm255985-9_4seq1.xml,OWNERSHIP DOCUMENT,CAT,18230
4,0001104659-25-012853,2025-02-13,2025-02-11,2025-02-13T14:39:50.000Z,NaN,4,NaN,NaN,NaN,4,6513,0,0,xslF345X05/tm255985-8_4seq1.xml,OWNERSHIP DOCUMENT,CAT,18230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,0000018230-17-000191,2017-06-29,2016-12-31,2017-06-29T14:57:04.000Z,34,11-K,001-00768,17937956.0,NaN,11-K,470489,0,0,caterpillarglobalminingleg.htm,"11-K, GLOBAL MINING HOURLY PLAN",CAT,18230
1003,0000018230-17-000190,2017-06-29,2016-12-31,2017-06-29T14:50:39.000Z,34,11-K,001-00768,17937930.0,NaN,11-K,466285,0,0,solarsavingsinvestmentplan.htm,"11-K, SOLAR INVESTMENT PLAN",CAT,18230
1004,0000018230-17-000189,2017-06-29,2016-12-31,2017-06-29T14:45:40.000Z,34,11-K,001-00768,17937915.0,NaN,11-K,465253,0,0,taxdeferredretirementplan2.htm,"11-K, TAX DEFERRED RETIREMENT PLAN",CAT,18230
1005,0000018230-17-000188,2017-06-29,2016-12-31,2017-06-29T14:41:41.000Z,34,11-K,001-00768,17937899.0,NaN,11-K,480889,0,0,a401ksavingsplan2016.htm,"11-K, 401K SAVINGS PLAN",CAT,18230
